In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
hs_train=pd.read_csv("../input/av-healthcare-analytics-ii/healthcare/train_data.csv")
hs_test=pd.read_csv("../input/av-healthcare-analytics-ii/healthcare/test_data.csv")

In [ ]:
hs_train.drop(['case_id'],axis=1,inplace=True)
hs_test.drop(['case_id'],axis=1)

In [ ]:
for feature in hs_train.columns: 
    if hs_train[feature].dtype == object:
        print('\n')
        print('feature:',feature)
        print(pd.Categorical(hs_train[feature].unique()))

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [ ]:
hs_train.groupby(by=['Department'])['patientid'].sum().reset_index().sort_values(['patientid'],
                    ascending=False).head(20).plot(x='Department',y='patientid',kind='bar', figsize=(15,5))

plt.show()

In [ ]:
plt.figure(figsize=(20,5))
sns.countplot(hs_train['Stay'], order = hs_train['Stay'].value_counts().index)

In [ ]:
hs_train['Stay'].value_counts(normalize=True)

In [ ]:
hs_train['Bed Grade'].unique()

In [ ]:
sns.countplot(hs_train['Bed Grade'], order = hs_train['Bed Grade'].value_counts().index)

In [ ]:
hs_train.isnull().sum().sort_values(ascending = False)/hs_train.index.size*100

In [ ]:
hs_test.isnull().sum().sort_values(ascending = False)/hs_test.index.size*100

In [ ]:
hs_train['City_Code_Patient'].fillna(hs_train['City_Code_Patient'].median(),inplace=True)
hs_train['Bed Grade'].fillna(hs_train['Bed Grade'].median(),inplace=True)

In [ ]:
hs_test['City_Code_Patient'].fillna(hs_test['City_Code_Patient'].median(),inplace=True)
hs_test['Bed Grade'].fillna(hs_test['Bed Grade'].median(),inplace=True)

In [ ]:
hs_train.describe()

In [ ]:
hs_test.describe()

In [ ]:
cont_train=hs_train.drop(['Hospital_type_code','Hospital_region_code','Department','Ward_Type','Ward_Facility_Code','Type of Admission','Severity of Illness','Age','Stay'],axis=1)
cont_test=hs_test.drop(['Hospital_type_code','Hospital_region_code','Department','Ward_Type','Ward_Facility_Code','Type of Admission','Severity of Illness','Age'],axis=1)

In [ ]:
for i in cont_train:
    sns.boxplot(cont_train[i])
    plt.show()

In [ ]:
def remove_outlier(col):
    sorted(col)
    Q1,Q3=np.percentile(col,[25,75])
    IQR=Q3-Q1
    lower_range= Q1-(1.5 * IQR)
    upper_range= Q3+(1.5 * IQR)
    return lower_range, upper_range

In [ ]:
for column in cont_test.columns:
    lr,ur=remove_outlier(cont_test[column])
    cont_test[column]=np.where(cont_test[column]>ur,ur,cont_test[column])
    cont_test[column]=np.where(cont_test[column]<lr,lr,cont_test[column])

In [ ]:
for i in cont_test:
    sns.boxplot(cont_test[i])
    plt.show()

In [ ]:
stay_dict = {'0-10': 0, '11-20': 1, '21-30': 2, '31-40': 3, '41-50': 4, '51-60': 5, '61-70': 6, '71-80': 7, '81-90': 8, '91-100': 9, 'More than 100 Days': 10}

In [ ]:
hs_train['Stay'] = hs_train['Stay'].replace(stay_dict.keys(), stay_dict.values())

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder() 

In [ ]:
hs_train['Hospital_type_code'] = labelencoder.fit_transform(hs_train.Hospital_type_code)
hs_train['Hospital_region_code'] = labelencoder.fit_transform(hs_train.Hospital_region_code)
hs_train['Department'] = labelencoder.fit_transform(hs_train.Department)
hs_train['Ward_Type'] = labelencoder.fit_transform(hs_train.Ward_Type)
hs_train['Ward_Facility_Code'] = labelencoder.fit_transform(hs_train.Ward_Facility_Code)
hs_train['Type of Admission'] = labelencoder.fit_transform(hs_train['Type of Admission'])
hs_train['Severity of Illness'] = labelencoder.fit_transform(hs_train['Severity of Illness'])
hs_train['Age'] = labelencoder.fit_transform(hs_train.Age)

In [ ]:
hs_test['Ward_Type'] = labelencoder.fit_transform(hs_test.Ward_Type)
hs_test['Ward_Facility_Code'] = labelencoder.fit_transform(hs_test.Ward_Facility_Code)
hs_test['Type of Admission'] = labelencoder.fit_transform(hs_test['Type of Admission'])
hs_test['Severity of Illness'] = labelencoder.fit_transform(hs_test['Severity of Illness'])
hs_test['Age'] = labelencoder.fit_transform(hs_test.Age)
hs_test['Hospital_type_code'] = labelencoder.fit_transform(hs_test.Hospital_type_code)
hs_test['Hospital_region_code'] = labelencoder.fit_transform(hs_test.Hospital_region_code)
hs_test['Department'] = labelencoder.fit_transform(hs_test.Department)

In [ ]:
x_train = hs_train.drop('Stay', axis = 1)
y_train = hs_train['Stay']

In [ ]:
x_test=hs_test

In [ ]:
x_test.head()

In [ ]:
hs_test.drop(['case_id'],axis=1,inplace=True)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
RF_model=RandomForestClassifier(n_estimators=300, random_state=1)
RF_model.fit(x_train, y_train)

In [ ]:
RF_model.score(x_train, y_train)

In [ ]:
y_pred=RF_model.predict(x_test)

In [ ]:
RF_model.score(x_test,y_pred)

In [ ]:
hs_test_new=pd.read_csv("../input/av-healthcare-analytics-ii/healthcare/test_data.csv")

In [ ]:
Result=pd.DataFrame({'Case_id':hs_test_new["case_id"],
                    "Stay":y_pred})

In [ ]:
Result['Stay'] = Result['Stay'].replace(stay_dict.values(),stay_dict.keys())

In [ ]:
Result